## Wykład 13 - Kotlin `Coroutines`

Coroutines pozwalają na pisanie kodu asynchronicznego w sposób synchroniczny, dzięki czemu jest on łatwiejszy do zrozumienia i pisania. Coroutines są oparte na idei "zawieszania" i "wznawiania" wykonania, co pozwala na **unikanie blokowania wątków**. Coroutines można wykorzystać do różnych zadań, takich jak obsługa sieci, wczytywanie danych z pliku lub bazy danych, itp.

Jest to koncepcja (**continuations**) - abstrakcja opisująca aktualny stan programu (wątku) w sposób umożliwiający wznowienie działania po zawieszeniu.

##  Funkcje zawieszone - `suspend fun`

Funkcje zawieszone (`suspend fun`) są to funkcje ze zdolnością do zawieszenia wykonania kodu i wznowienia w późniejszym czasie.

Gdy wykonanie jest zawieszone, kompilator zapisuje stan wraz ze wszystkimi niezbędnymi informacjami (przykładowo lokalne zmienne) - powstały obiekt jest niewielki. Jeżeli funkcja jest zawieszona, inna funkcja może zostać wykonana na danym wątku.

**Nie występuje sytuacja zablokowania wątku**

```kotlin
suspend fun delay(timeMillis: Long) { ... }
```

Aby korzystać z `Coroutines` należy dodać zależność

In [ ]:
dependencies {
    implementation("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.6.4")
    ...
}

## Budowniczy - `Coroutine.Builder`

Aby wykonać funkcje zawieszoną musi ona zostać umieszczona w odpowiednim kontenerze - tym kontenerem jest `Coroutine`. Każda `Coroutine` musi zostać wykonana na wątku - ale nie musi to być jeden określony wątek. Przykłady:
- `launch` - gdy nie interesuje nas wartość zwracana
- `async`
- `runBlocking` - gdy chcemy zablokować wykonanie wątku

In [ ]:
import kotlinx.coroutines.*

fun main() {
    GlobalScope.launch { // launch a new coroutine in background and continue
        delay(1000L) // non-blocking delay for 1 second (default time unit is ms)
        println("World!") // print after delay
    }
    println("Hello,") // main thread continues while coroutine is delayed
    Thread.sleep(2000L) // block main thread for 2 seconds to keep JVM alive
}

`launch` tworzy `Coroutine` i rozpoczyna wykonanie wyrażenia lambda. Wpierw wykonujemy `delay` aby zawiesić wykonanie. W tym samym czasie kod poza `Coroutine` zostaje wykonany bez blokowania. Gdy nie użyjemy `Thread.sleep` funkcja `main`, a tym samym JVM, zakończy działanie.

Kolejną opcją jest `runBlocking`, który tworzy `Coroutine` i blokuje aktualny wątek dopóki utworzona `Coroutine` nie zakończy działania

In [ ]:
fun main() {
    runBlocking {
        delay(1000)
        println("Hello World")
    }
    println("Done")
}

In [ ]:
fun main() {
    suspend fun a() {
        delay(100)
        println("a")
    }
    suspend fun b() {
        delay(100)
        println("b")
    }
    runBlocking {
        a()
        b()
        println("Done")
    }
}


In [ ]:
fun main() {
    suspend fun a(): Int = {
        delay(100)
        Random.nextInt()
    }
    suspend fun b(a: Int) {
        println("Using $a")
    }
    runBlocking {
        val a = a()
        b(a)
    }
}


In [ ]:
fun main() {
    runBlocking {
        println("Parent starting")
        launch {
            println("Child a starting")
            delay(100)
            println("Child a complete")
        }
        launch {
            println("Child b starting")
            delay(100)
            println("Child b complete")
        }
        println("Parent complete")
    }
    println("Parent returned")
}

## `Job`

Gdy `Coroutine` jest utworzony przy wykorzystaniu `launch`, zwracana jest wartość typu `Job`. Jedną z głównych funkcji `Job` jest możliwość zakończenia wykonania.

In [ ]:
fun main() {
    var x = 0
    val job = GlobalScope.launch {
        while (true) {
            x += 1
            delay(10)
            println(x)
        }
    }
    runBlocking {
        delay(100)
        job.cancel()
    }
}

Następną jest możliwość oczekiwania na zakończenie działania za pomocą metody `join`.

In [ ]:
fun main() {
    val job = GlobalScope.launch {
        delay(1000L)
        println("Hello")
    }
    runBlocking {
        job.join()
        println("World")
    }
}

In [ ]:
fun main() = runBlocking {
    repeat(100) { // launch a lot of coroutines
        launch {
            delay(5000L)
            print(".")
        }
    }
}

## `Context`

Każda `Coroutine` posiada powiązany `Context` - jest to niemutowalna kolekcja kluczy i wartości - dzięki temu możemy nadać nazwę każdej `Coroutine` lub określić który wątek z powiązanego `ThreadPool` powinien wykonać zadanie.

Kontekst w Coroutine jest to zestaw obiektów, które są dostępne dla Coroutine podczas jej działania. Kontekst może zawierać informacje takie jak wątek, który jest używany do wykonania Coroutine, czy też dane, które są przechowywane w koroutynie.

Coroutine mogą być wykonywane na różnych wątkach za pomocą kontekstu. Na przykład, Coroutine może być uruchamiana na wątku interfejsu użytkownika za pomocą kontekstu `Dispatchers.Main`, lub na wątku domyślnym za pomocą kontekstu `Dispatchers.Default`. Możemy też stworzyć własny kontekst dla Coroutine, na przykład za pomocą `newSingleThreadContext("MyThread")`.

Kontekst jest również ważny dla obsługi błędów i anulowania Coroutine. Gdy Coroutine jest anulowana, wszystkie Coroutine, które są uruchomione w tym samym kontekście są również anulowane.

## `Dispatcher`

Jeżeli chcemy mieć kontrolę nad tym który wątek z puli dana `Coroutine` jest wykonana możemy wykorzystać `Dispatcher`
- `Dispatcher.Default` - domyślny wykorzystuje `ThreadPool`, którego wielkość jest równa liczbie rdzeni CPU
- `Dispatcher.IO` - rekomendowany do operacji IO, wykorzystuje `ThreadPool` który może zostać rozszerzony
- `Dispatcher.Main` - wykorzystuje wątek główny UI.

In [ ]:
fun main() {
    runBlocking(Dispatchers.IO) {
        println(Thread.currentThread().name)
    }
}

## `Async`

In [ ]:
fun main() {
    runBlocking {
        val deferred = async {
            println("Computing value...")
            delay(1000)
            10
        }
        val result = deferred.await()
        println("The result is $result")
    }
}

In [ ]:
import kotlinx.coroutines.*
import kotlin.system.measureTimeMillis

fun main() {
    suspend fun fetchOrder(): String {
        delay(3000)
        return "Order 123"
    }
    suspend fun fetchAddress(): String {
        delay(2000)
        return "10 Downing Street"
    }
    suspend fun fetchUsername(): String {
        delay(1000)
        return "Prime Minister"
    }
    println("Starting")
    val time = measureTimeMillis {
        runBlocking {
            val order = async {
                fetchOrder()
            }
            val address = async {
                fetchAddress()
            }
            val username = async {
                fetchUsername()
            }
            println("Shipping ${order.await()} to ${username.await()}at ${address.await()}")
        }
    }
    println("Completed in $time ms")
}


## 14.6 `Dispatcher`

Jeżeli chcemy mieć kontrolę nad tym który wątek z puli dana `Coroutine` jest wykonana możemy wykorzystać `Dispatcher`
- `Dispatcher.Default` - domyślny wykorzystuje `ThreadPool`, którego wielkość jest równa liczbie rdzeni CPU
- `Dispatcher.IO` - rekomendowany do operacji IO, wykorzystuje `ThreadPool` który oże zostać rozszerzony
- `Dispatcher.Main` - wykorzystuje wątek główny UI.

In [ ]:
fun main() {
    runBlocking(Dispatchers.IO) {
        println(Thread.currentThread().name)
    }
}

## `Coroutines` vs `Executor`

In [ ]:
val sum = 17
fun main() {
    val executor = Executors.newSingleThreadExecutor()
    val callable: Callable<BigInteger> = Callable {
        var result = BigInteger.ZERO
        for (i in 0..sum) {
            result = result.add(BigInteger.valueOf(sum.toLong()))
        }
        result
    }
    val taskFuture = executor.submit(callable)
    try {
        while (!taskFuture.isDone) println("waiting")
        println(taskFuture.get())
    } catch (ee: ExecutionException) {
        System.err.println("task threw an exception")
        ee.printStackTrace()
    } catch (ie: InterruptedException) {
        System.err.println("interrupted while waiting")
    }
    executor.shutdown()
}

In [ ]:
import kotlinx.coroutines.*
import java.math.BigInteger

val sum = 17

fun main() {
    runBlocking {
        val result = async {
            var result = BigInteger.ZERO
            for (i in 0..sum) {
                result = result.add(BigInteger.valueOf(i.toLong()))
            }
            result
        }
        println("waiting")
        println(result.await())
    }
}


In [ ]:
import java.math.BigInteger;
import java.util.ArrayList;
import java.util.List;
import java.util.concurrent.*;

public class Main {
    private static BigInteger calculatedSum = BigInteger.ZERO;
    private static final int NUM_THREADS = 10;

    static class Sum implements Callable<BigInteger> {
        long from, to;
        BigInteger localSum = BigInteger.ZERO;

        public Sum(long from, long to) {
            this.from = from;
            this.to = to;
        }

        @Override
        public BigInteger call() {
            for (long i = from; i <= to; i++)
                localSum = localSum.add(BigInteger.valueOf(i));
            return localSum;
        }
    }

    public static void main() {
        ExecutorService executorService = Executors.newFixedThreadPool(NUM_THREADS);
        List<Future<BigInteger>> summationTasks = new ArrayList<>();

        long n = 1000000L;
        long taskNum = n / 10; // 10 tasks

        for (int i = 0; i < NUM_THREADS; i++) {
            long fromInInnerRange = (taskNum * i) + 1;
            long toInInnerRange = taskNum * (i + 1);

            System.out.printf("Wątek dla sumy w zakresie %d - %d %n", fromInInnerRange, toInInnerRange);
            Callable<BigInteger> summationTask = new Sum(fromInInnerRange, toInInnerRange);
            Future<BigInteger> futureSum = executorService.submit(summationTask);
            summationTasks.add(futureSum);
        }

        for (Future<BigInteger> partialSum : summationTasks) {
            try {
                calculatedSum = calculatedSum.add(partialSum.get());
            } catch (CancellationException | ExecutionException | InterruptedException exception) {
                exception.printStackTrace();
            }
        }
        System.out.printf("Suma = %d", calculatedSum);
        executorService.shutdown();
    }
}

In [ ]:
private var calculatedSum = BigInteger.ZERO
private const val NUM_THREADS = 10

class Sum(private val from: Long, private val to: Long) 
    : CoroutineScope by CoroutineScope(Dispatchers.Default) {
    private var localSum: BigInteger = BigInteger.ZERO
    fun call(): BigInteger {
        for (i in from..to) {
            localSum = localSum.add(BigInteger.valueOf(i))
        }
        return localSum
    }
}

fun main() {
    runBlocking {
        val summationTasks = mutableListOf<Deferred<BigInteger>>()
        val n = 1000000L
        val taskNum = n / NUM_THREADS
        for (i in 0 until NUM_THREADS) {
            val fromInInnerRange = (taskNum * i) + 1
            val toInInnerRange = taskNum * (i + 1)
            println("Coroutine dla sumy w zakresie $fromInInnerRange - $toInInnerRange")

            val summationTask = async { Sum(fromInInnerRange, toInInnerRange).call() }
            summationTasks.add(summationTask)
        }
        for (partialSum in summationTasks) {
            calculatedSum = calculatedSum.add(partialSum.await())
        }
        println("Suma = $calculatedSum")
    }
}

W powyższym kodzie, używamy `async` z `Coroutine` zamiast `ExecutorService` i `Future` z wątkami. Tworzymy klasę `Sum` która jest rozszerzeniem `CoroutineScope` i posiada dwie wartości typu `long` - `from` i `to`, które są początkowym i końcowym zakresem sumowania. Klasa ta posiada `fun call`, która sumuje liczby od `from` do `to` i zwraca wynik.
W funkcji `main`, tworzymy listę obiektów `Deferred<BigInteger>` i za pomocą pętli `for` tworzymy 10 `Coroutine` za pomocą `async` i dodajemy je do listy. Następnie przy pomocy `await()` czekamy na zakończenie `Coroutine` i sumujemy wynik.

Ważne jest aby pamiętać, że w przeciwieństwie do wątków, `Coroutine` są automatycznie zarządzane przez system i nie jest konieczne ręczne zamykanie ich przy pomocy `executorService.shutdown()`.

`ExecutorService` i `Coroutines` to dwa różne sposoby na wykonywanie zadań w tle.

`ExecutorService` to interfejs Java, który pozwala na uruchamianie zadań w tle za pomocą wątków. Zadania mogą być przesłane do `ExecutorService` jako obiekty implementujące interfejs `Callable` lub `Runnable`. `ExecutorService` udostępnia również funkcjonalności takie jak zarządzanie pulą wątków i planowanie zadań.

`Coroutines` to mechanizm Kotlin, który również pozwala na wykonywanie zadań w tle. Pozwalają na bardziej efektywne zarządzanie pamięcią i mniejsze opóźnienia niż wątki, są również łatwiejsze w użyciu niż wątki, ponieważ pozwalają na użycie zwykłych funkcji zamiast implementacji interfejsów.

Ogólnie, `ExecutorService` jest bardziej uniwersalnym rozwiązaniem, które jest dostępne dla Java i innych języków, które mogą korzystać z Java SDK. `Coroutines` natomiast są specjalnie dostosowane do Kotlina i oferują bardziej łatwiejszą składnię oraz lepsze zarządzanie zadaniami w tle. Dlatego w zależności od potrzeb projektu i preferencji developera, można wybrać odpowiednie rozwiązanie dla swoich potrzeb.